In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
input_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [ ]:
def pre_process(df, drop_features, output_col):
    out = df[output_col]
    df = df.drop(drop_features, axis = 1)
    return df, out

In [ ]:
X, y = pre_process(input_df, ['location', 'id', 'keyword', 'target'], 'target')

In [ ]:
print(X.head())
print(y)

In [ ]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define Transformer block
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
# class TransformerBlock(layers.Layer):
#     def _init_(self, embed_dim, num_heads, ff_dim, rate=0.1):
#         super()._init_()
#         self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
#         self.ffn = keras.Sequential(
#             [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
#         )
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout1 = layers.Dropout(rate)
#         self.dropout2 = layers.Dropout(rate)

#     def call(self, inputs):
#         attn_output = self.att(inputs, inputs)
#         attn_output = self.dropout1(attn_output)
#         out1 = self.layernorm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output)
#         return self.layernorm2(out1 + ffn_output)

In [ ]:
# Define hyperparameters
vocab_size = 10000
embedding_dim = 128
max_seq_length = 100
num_heads = 2
ff_dim = 128
num_classes = 2

# Define the Transformer model
def transformer_model(vocab_size, embedding_dim, max_seq_length, num_heads, ff_dim, num_classes):
    inputs = Input(shape=(max_seq_length,))
    embedding_layer = Embedding(vocab_size, embedding_dim)(inputs)
    transformer_block = TransformerBlock(embedding_dim,
        num_heads,
        ff_dim,
        0.1
    )
    transformer_output = transformer_block(embedding_layer, training=True)
    pooling_layer = GlobalAveragePooling1D()(transformer_output)
    outputs = Dense(num_classes, activation='softmax')(pooling_layer)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def get_sequences(texts):
    # Initialize tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)

    # Convert texts to sequences of integers
    return tokenizer.texts_to_sequences(texts)

In [ ]:
# Instantiate the model
model = transformer_model(vocab_size, embedding_dim, max_seq_length, num_heads, ff_dim, num_classes)
model.compile(optimizer=Adam(learning_rate=1e-4), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Define some example data (replace this with your actual dataset)
# Example data
X_train = pad_sequences(get_sequences(X_train['text']), maxlen=max_seq_length)
X_test = pad_sequences(get_sequences(X_test['text']), maxlen=max_seq_length)

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test), callbacks=[EarlyStopping(patience=3)])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
model.save('/kaggle/working/disaster_tweets_model.keras')

In [ ]:
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
test_X = test_df.drop(['location', 'id', 'keyword'], axis = 1)

In [ ]:
test_X = pad_sequences(get_sequences(test_X['text']), maxlen=max_seq_length)

In [ ]:
test_X

In [ ]:
predictions = model.predict(test_X)

In [ ]:
binary_predictions = (predictions > 0.5).astype(int)

In [ ]:
test_df.head(2)

In [ ]:
predictions

In [ ]:
predictions_arr = []
for i in range(len(binary_predictions)):
    predictions_arr.append(binary_predictions[i][0])

In [ ]:
len(predictions_arr)

In [ ]:
len(test_df)

In [ ]:
ans_df = pd.DataFrame({
    "id": test_df["id"],
    "target": predictions_arr
})

In [ ]:
ans_df.head()

In [ ]:
ans_df.to_csv("/kaggle/working/submission.csv", index=False)